<a href="https://colab.research.google.com/github/syedaahmed05/Quest-2/blob/main/PIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST Hand Written digit recognition

## Importing necessary packages

In [ ]:
# Importing necessary packages
import torch
from torch import nn

# Copmuter vision modules
import torchvision
import torchmetrics
from torchvision import datasets
from torchvision.transforms import ToTensor


# Import matplotlib for visualization
import matplotlib.pyplot as plt

import tqdm



In [ ]:
!pip install torchmetrics

In [ ]:
#checking pytorch versions

print(f'PyTorch version: {torch.__version__} \ntorchvision version: {torchvision.__version__}')

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

## MNIST Dataset

Stands for Modified National Institure of Standards and Technology

> * The data we will be working with has examples of handwritten digits (from 0 to 9)

In [ ]:
# Setup training data

train_data = datasets.MNIST(
                            root="data", # where should we download the data to
                            train = True, # Use data for training
                            download =True, # Download data
                            transform = ToTensor(), # Data is in PIL format we need tensors for training
                            target_transform = None # for transforming labels
)

# Setup testing Data

test_data = datasets.MNIST(
                            root = "data",
                            train = False,
                            download = True,
                            transform = ToTensor()
)




In [ ]:
# Explore the data
image, label = train_data[1]

# Visualize the image
plt.imshow(image.squeeze(), cmap="gray")
plt.title(f"Label: {label}")
plt.show()

print(f"Image Shape ={image.shape}\nLabel  = {label}")

In [ ]:
# More about the values within these tensors
max_value = image.max().item()

min_value = image.min().item()

print(f"Max value: {max_value}")
print(f"Min value: {min_value}")

In [ ]:
# Example of the values within the tensor
image[:,0,0], image[:,23,12]


In [ ]:
# How many samples do we have in the training and testing data
len(train_data.data), len(train_data.targets), len(test_data.data), len(test_data.targets)

In [ ]:
# Classes within the dataset
class_names = train_data.classes
class_names

In [ ]:
# Plot one image from each class

torch.manual_seed(12)
fig = plt.figure(figsize=(10,5))
rows, cols = 2, 5  # Adjusted to fit 10 classes

for i, class_name in enumerate(class_names):
    for img, label in train_data:
        if class_names[label] == class_name:
            fig.add_subplot(rows, cols, i+1)
            plt.title(class_name)
            plt.axis("off")
            plt.imshow(img.squeeze(), cmap="gray")
            break

plt.show()

## Data Preperation

In [ ]:
from torch.utils.data import DataLoader

#Setup batch size
BATCH_SIZE = 32

#Turn datasets into iterables

train_dataloader = DataLoader(train_data, # turne training data into iterable
                                batch_size = BATCH_SIZE, # set batch size
                                shuffle = True # shuffle data)
                                )

test_dataloader = DataLoader(test_data, # turne testing data into iterable
                                batch_size = BATCH_SIZE, # set batch size
                                shuffle = False # shuffle data)
                                )

# Check the changes the made

print(f"Length of train dataloader: {len(train_dataloader)} batches of size {BATCH_SIZE}")

print(f"Length of test dataloader: {len(test_dataloader)} batches of size {BATCH_SIZE}")

## Building a model

### Base model

In [ ]:
class SimpleModel(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
       super().__init__() #   Call the __init__ method of the parent class
       self.layer_stack = nn.Sequential(
           nn.Flatten(), # convert the input from 28x28 to 784 (Matrix to vector)
           nn.Linear(in_features=input_shape, out_features=hidden_units),
           nn.Linear(in_features=hidden_units, out_features=output_shape)
       )

    def forward(self, x):
        return self.layer_stack(x)


In [ ]:
torch.manual_seed(12)

# Define the input shape, hidden units, and output shape
model_0 = SimpleModel(input_shape=28*28, # 28x28 image
                        hidden_units=10, # No of neurons in the hidden layer
                         output_shape=len(class_names)) # Output shape

model_0.to(device) # Move the model to the GPU if available

### Importing metrics

In [ ]:
# Accuracy metric

metric = torchmetrics.classification.Accuracy(task='multiclass',num_classes=len(class_names)).to(device)


In [ ]:
# Import tqdm for progress bar
from tqdm.auto import tqdm

# Set the seed and start the timer
torch.manual_seed(12)


# Set the number of epochs (we'll keep this small for faster training times)
epochs = 5

def train_step(model: torch.nn.Module,
               epochs: int = 5,  # Default epochs
               data_loader: torch.utils.data.DataLoader = train_dataloader,  # Default data loader
               loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),  # Default loss function
               optimizer: torch.optim.Optimizer = torch.optim.SGD, # Default optimizer class
               learning_rate: int = 0.003, # keyword arguments to pass into optimizer instantiation.
               accuracy_fn=metric,
               device: torch.device = device):

    # if optimizer is a class, instantiate with parameters from the model.
    if isinstance(optimizer, type):
        optimizer = optimizer(model.parameters(), learning_rate)

    train_loss, train_acc = 0, 0
    model.to(device)

    for epoch in tqdm(range(epochs)):
        print(f"Epoch: {epoch}\n-------")
        for batch, (X, y) in tqdm(enumerate(data_loader)):
            # Send data to GPU
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            y_pred = model(X)

            # 2. Calculate loss
            loss = loss_fn(y_pred, y)
            train_loss += loss
            acc = metric(y_pred,y) # Go from logits -> pred labels
            acc = metric.compute()
            train_acc += acc
            metric.reset()

            # 3. Optimizer zero grad
            optimizer.zero_grad()

            # 4. Loss backward
            loss.backward()

            # 5. Optimizer step
            optimizer.step()

        # Calculate loss and accuracy per epoch and print out what's happening
        train_loss /= len(data_loader)
        train_acc /= len(data_loader)
        print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

In [ ]:
def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
              accuracy_fn = metric,
              device: torch.device = device):
    test_loss, test_acc = 0, 0
    model.to(device)
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode():
        for X, y in tqdm(data_loader):
            # Send data to GPU
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred = model(X)

            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y)
            acc = metric(test_pred,y) # Go from logits -> pred labels
            acc = metric.compute()
            test_acc += acc
            metric.reset()
        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

### Evaluvate model_0

In [ ]:
torch.manual_seed(12)

def eval_model(model:torch.nn.Module,
                data_loader:torch.utils.data.DataLoader,
                loss_fn:torch.nn.Module,
                metric:torchmetrics.Metric):
    """
        Returns a dictionary containing the results of model predicting on data_loader.

        Args:
            model (torch.nn.Module): A PyTorch model capable of making predictions on data_loader.
            data_loader (torch.utils.data.DataLoader): The target dataset to predict on.
            loss_fn (torch.nn.Module): The loss function of model.
            accuracy_fn: An accuracy function to compare the models predictions to the truth labels.

        Returns:
            (dict): Results of model making predictions on data_loader.
    """

    loss, acc =0, 0

    model.eval()

    with torch.inference_mode():
        for X, y in data_loader:
            # Setting tensors to device
            X = X.to(device)
            y = y.to(device)

            # Making predictions
            y_pred = model(X)

            # Claculating loss
            loss += loss_fn(y_pred, y)

            # Calculating accuracy
            acc += metric(y_pred, y)
            metric.reset()

        # Calculate average loss and accuracy
        loss /= len(data_loader)
        acc = acc / len(data_loader) * 100

        return{"Model_name":model.__class__.__name__,
                "Loss":loss,
                "Accuracy":acc}

# caluclate the evaluation metrics
model_0_results = eval_model(model=model_0,
                            data_loader=test_dataloader,
                            loss_fn= nn.CrossEntropyLoss(),
                            metric=metric)

model_0_results




## Build a model with non-linearity

In [ ]:
class Model_1(nn.Module):
    def __init__(self, input_shape: int, hidden_layer_1: int, hidden_layer_2: int, output_shape: int)->None:
      super().__init__()
      self.layer_stack = nn.Sequential(
          nn.Flatten(),
          nn.Linear(in_features=input_shape,out_features=hidden_layer_1),
          nn.ReLU(),
          nn.Linear(in_features=hidden_layer_1, out_features=hidden_layer_2),
          nn.ReLU(),
          nn.Linear(in_features=hidden_layer_2, out_features=10),
      )

    def forward(self, x):
        return self.layer_stack(x)




In [ ]:
torch.manual_seed(12)

model_1 = Model_1(input_shape=28*28,
                     hidden_layer_1=10,
                     hidden_layer_2=15,
                     output_shape=len(class_names)
                     ).to(device)

#checking model 1
next(model_1.parameters()).device

In [ ]:
train_step(model=model_1,
          epochs = 7,
          data_loader= train_dataloader,
          optimizer = torch.optim.Adam,
          learning_rate=0.005,
          accuracy_fn=metric,
          device=device)

In [ ]:
model_1_results = eval_model(model=model_1,
                            data_loader=test_dataloader,
                            loss_fn= nn.CrossEntropyLoss(),
                            metric=metric)

model_1_results

## Build model with CNN

In [ ]:
# Create a convolutional neural network
class CNN_module(nn.Module):
    """
    Model architecture copying TinyVGG from:
    https://poloclub.github.io/cnn-explainer/
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3, # how big is the square that's going over the image?
                      stride=1, # default
                      padding=1),# options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) # default stride value is same as kernel_size
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            # Where did this in_features shape come from?
            # It's because each layer of our network compresses and changes the shape of our input data.
            nn.Linear(in_features=hidden_units*7*7,
                      out_features=output_shape)
        )

    def forward(self, x: torch.Tensor):
        x = self.block_1(x)
        # print(x.shape)
        x = self.block_2(x)
        # print(x.shape)
        x = self.classifier(x)
        # print(x.shape)
        return x

torch.manual_seed(42)
model_2 = CNN_module(input_shape=1,
    hidden_units=10,
    output_shape=len(class_names)).to(device)
model_2

### Training Model

In [ ]:
train_step(model=model_2,
          epochs = 7,
          data_loader= train_dataloader,
          optimizer = torch.optim.Adam,
          learning_rate=0.003,
          accuracy_fn=metric,
          device=device)

### Evaluate model

In [ ]:
eval_model(model=model_2,
            data_loader=test_dataloader,
            loss_fn= nn.CrossEntropyLoss(),
            metric=metric)

# Your Turn!!

> Try to make the best model messing with the

In [ ]:
train_step(model = model_0, # we have built 3 models so far, choose which one you would like to change
               epochs = 5,  # Default epochs (Larger numbers might take more time)
               data_loader= train_dataloader,
               loss_fn =  nn.CrossEntropyLoss(),  # Default loss function
               optimizer = torch.optim.SGD, # Default optimizer class
               learning_rate =  0.003, # keyword arguments to pass into optimizer instantiation.
               accuracy_fn=metric,
               device = device):

reslults = eval_model(model=model_0,
            data_loader=test_dataloader,
            loss_fn= nn.CrossEntropyLoss(),
            metric=metric)

print(results)

"""
loss functions : https://pytorch.org/docs/stable/nn.html#loss-functions
optimizers : https://pytorch.org/docs/stable/optim.html


"""